### (1) Import the necessary packages

In [2]:
import csv
import numpy as np

### (2) Read temperature_data.csv. Note that temperature_data.csv must be in the same directory as this Jupyter notebook. We store the data from temperature_data.csv in a [dictionary](https://docs.python.org/3/tutorial/datastructures.html?highlight=dictionary#dictionaries) called "data". 

In [3]:
# initialize the "data" dictionary
data = {"days": [],         # days
        "weather": [],      # weather.com prediction
        "accuweather": [],  # accuweather.com prediction
        "wunderground": [], # wunderground.com prediction
        "actual": []}       # actual temperature

# read the data from temperature_data.csv and store it in the dictionary called "data"
with open('temperature_data.csv', newline='') as f:
    reader = csv.reader(f)
    for idx, row in enumerate(reader):
        if idx > 0:
            data['days'].append(int(row[0]))
            data['weather'].append(float(row[1]))
            data['accuweather'].append(float(row[2]))
            data['wunderground'].append(float(row[3]))
            data['actual'].append(float(row[4]))

# print the "data" dictionary
#print(f"data = {data}")

### (3) Implement the weighted majority algorithm (WMA)

In [81]:
# Input Parameters
n = 3  # number of experts
T = 61  # time horizon
eps = 0.13  # parameter epsilon


# Initialization
w_exp = [1,1,1]  # numpy array of the experts' weights
m_exp = [0,0,0]  # numpy array of the number of mistakes made by experts
m_wma = 0  # the number of mistakes made by WMA

# Run the weighted majority algorithm
for t in range(1, T+1):
    # get experts' predictions for day t from the data dictionary in (2)
    predwe = int(data['weather'][t-1]<=70) #1 = below 70, 0 = above 70
    preda = int(data['accuweather'][t-1]<=70)
    predwu = int(data['wunderground'][t-1]<=70)
    preds = [predwe, preda, predwu]
    #print("Experts' predictions for day ", t, " are ", preds)
    
    # predict the outcome for day t according to WMA
    predtotal = preds[0]*w_exp[0] + preds[1]*w_exp[1] + preds[2]*w_exp[2]
    if predtotal >= (2/3)*sum(w_exp):
          outcome = 1
    else: outcome = 0 
    #print("Predicted Outcome: ", outcome)
    
    # get the true outcome for day t from the data dictionary in (2)
    actual = int(data['actual'][t-1]<=70)
    #print("Actual Outcome: ", actual)
    
    # update the experts' weights (w_exp) according to WMA
    for i in range(n):
        if (preds[i] != actual):
            m_exp[i] += 1
            w_exp[i] *= (1-eps)
    
    # update the number of mistakes made by experts (m_exp)^^ and WMA (m_wma)
    if (outcome != actual):
        m_wma += 1
    ##print("Mistakes made by WMA: ", m_wma, "Mistakes made by experts: ", m_exp)
    
    # print the experts' weights (w_exp) after round t
    print("Current experts' weights for day ", t, ": ", w_exp)

# print the number of mistakes made by WMA over T time periods
print("The total number of mistakes made by WMA: ", m_wma)

# print the number of mistakes made by the best expert in hindsight
print("The number of mistakes made by the best expert in hindsight", min(m_exp))

# print the (lower bound of - because of best expert) regret of WMA based on this single instance of temperatures
regret = m_wma - min(m_exp)
print("Regret = ", regret)

# Upper Mistake Bounds 
upperbest = 2*np.log(n)/eps + 2*min(m_exp)*(1+eps)
upperworst = 2*np.log(n)/eps + 2*max(m_exp)*(1+eps)

print("Upper bound of mistakes with best expert: ", upperbest) 
print("Upper bound of mistakes with worst expert: ", upperworst) 


1
1
1
1
1
1
1
1
1
1
1
1
1
0.87
0.87
0.7569
0.7569
0.7569
0.7569
0.7569
0.7569
0.6585030000000001
0.6585030000000001
0.6585030000000001
0.6585030000000001
0.6585030000000001
0.5728976100000001
0.5728976100000001
0.5728976100000001
0.49842092070000005
0.49842092070000005
0.433626201009
0.433626201009
0.37725479487783004
0.3282116715437121
0.28554415424302954
0.24842341419143568
0.21612837034654905
0.18803168220149769
0.18803168220149769
0.18803168220149769
0.16358756351530299
0.14232118025831358
0.14232118025831358
0.12381942682473282
0.10772290133751755
0.09371892416364026
0.09371892416364026
0.08153546402236703
0.08153546402236703
0.08153546402236703
0.07093585369945932
0.07093585369945932
0.07093585369945932
0.07093585369945932
0.07093585369945932
0.07093585369945932
0.061714192718529605
0.061714192718529605
0.05369134766512076
0.05369134766512076
The total number of mistakes made by WMA:  25
The number of mistakes made by the best expert in hindsight 20
Regret =  5
Upper bound of mis

In [77]:
##for AHHH in range(0,99): ## used for finding average number of mistakes by aglo 
### (4) Implement the randomized weighted majority algorithm (RWMA)
from random import random
from random import uniform

# Input Parameters
n = 3  # number of experts
T = 61  # time horizon
eps = 0.13  # parameter epsilon

# Initialization
w_exp = [1,1,1]  # numpy array of the experts' weights
m_exp = [0,0,0]  # numpy array of the number of mistakes made by experts
m_Rwma = 0  # the number of mistakes made by RWMA

# Run the R weighted majority algorithm
for t in range(1, T+1):
    # get experts' predictions for day t from the data dictionary in (2)
    predwe = int(data['weather'][t-1]<=70) #1 = below 70, 0 = above 70
    preda = int(data['accuweather'][t-1]<=70)
    predwu = int(data['wunderground'][t-1]<=70)
    preds = [predwe, preda, predwu]
    ##print("Experts' predictions for day ", t, " are ", preds)
    
    # predict the outcome for day t according to RWMA
    choice = random()*sum(w_exp)  # derived from https://jeremykun.com/tag/multiplicative-weights-update-algorithm/
    outcome_indx = 0
    for weight in w_exp:
        choice -= weight
        if choice <= 0:
            break
        else: outcome_indx += 1
    ##print('Predicted Outcome: ', preds[outcome_indx])

    # get the true outcome for day t from the data dictionary in (2)
    actual = int(data['actual'][t-1]<=70)
    ##print("Actual Outcome: ", actual)
    
    # update the experts' weights (w_exp) according to RWMA
    for i in range(n):
        if (preds[i] != actual):
            m_exp[i] += 1
            w_exp[i] *= (1-eps)
        
    # update the number of mistakes made by experts (m_exp)-done above and RWMA (m_Rwma)
    if (preds[outcome_indx] != actual):
        m_Rwma += 1
    ##print("Mistakes made by RWMA: ", m_Rwma, "Mistakes made by experts: ", m_exp) 
    
    # print the experts' weights (w_exp) after round t
    print("Current experts' weights for day ", t, ": ", w_exp)
    
# print the number of mistakes made by RWMA over T time periods
print("The total number of mistakes made by RWMA: ", m_Rwma)

# print the number of mistakes made by the best expert in hindsight
print("The number of mistakes made by the best expert in hindsight", min(m_exp))

# print the regret of RWMA based on this single instance of temperatures
regret = m_Rwma - min(m_exp)
print("Regret = ", regret)

# Upper Mistake Bounds 
upperbest = np.log(n)/eps + min(m_exp)*(1+eps)
upperworst = np.log(n)/eps + max(m_exp)*(1+eps)

print("Upper bound of mistakes with best expert: ", upperbest) 
##print("Upper bound of mistakes with worst expert: ", upperworst) 


Current experts' weights for day  1 :  [1, 1, 1]
Current experts' weights for day  2 :  [1, 1, 1]
Current experts' weights for day  3 :  [1, 1, 1]
Current experts' weights for day  4 :  [1, 1, 1]
Current experts' weights for day  5 :  [1, 0.87, 1]
Current experts' weights for day  6 :  [1, 0.87, 1]
Current experts' weights for day  7 :  [1, 0.87, 1]
Current experts' weights for day  8 :  [1, 0.87, 1]
Current experts' weights for day  9 :  [1, 0.87, 1]
Current experts' weights for day  10 :  [1, 0.87, 1]
Current experts' weights for day  11 :  [1, 0.87, 1]
Current experts' weights for day  12 :  [1, 0.87, 1]
Current experts' weights for day  13 :  [1, 0.87, 1]
Current experts' weights for day  14 :  [0.87, 0.7569, 0.87]
Current experts' weights for day  15 :  [0.87, 0.7569, 0.87]
Current experts' weights for day  16 :  [0.87, 0.7569, 0.7569]
Current experts' weights for day  17 :  [0.87, 0.7569, 0.7569]
Current experts' weights for day  18 :  [0.87, 0.6585030000000001, 0.7569]
Current e